## Step 1: Load Cleaned Dataset

We begin by loading the cleaned dataset produced during the EDA phase. This version includes standardized currency values, imputed missing data, and normalized text fields—ensuring consistency and reliability for rule-based categorization.


In [7]:
import pandas as pd
df = pd.read_excel("../data/2. cleaned-data-after-eda.xlsx", sheet_name="Sheet1")


---
## Step 2: Define Category Mapping Rules

To enable rule-based categorization, I created a dictionary of category labels mapped to relevant keywords in both English and Arabic. These keywords represent common terms found in item descriptions and are grouped based on domain knowledge and procurement patterns.

Each category is designed to capture a specific material or product type, such as construction metals, plumbing fixtures, electrical components, or safety gear. This structure allows for transparent, interpretable classification and supports multilingual matching.

- The `"Other"` category is reserved for items that do not match any predefined keyword set and will be handled separately during assignment.
- The `"Unknown"` category is for those who has Unknow Item named in the list
---

In [10]:
categories = {
    # Metals & Construction
    "Steel & Rebar": [
        "rebar", "steel bar", "حديد", "تسليح", "سابك", "راجحي", "ipe", "heb", "beam"
    ],
    "Sheets & Metals": [
        "gi sheet", "corrugated", "صاج", "لوح", "b.sheet", "plate", "coil", "galvanized", "sheet"
    ],
    
    # Concrete, Cement, Plaster
    "Cement & Gypsum": [
        "cement", "gypsum", "معجون", "جبس", "compound", "plaster", "putty"
    ],
    "Paints & Coatings": [
        "paint", "primer", "دهان", "epoxy", "coating", "varnish"
    ],
    "Wood & Panels": [
        "plywood", "mdf", "خشب", "panel board", "veneer", "timber"
    ],

    # Pipes, Plumbing, Sanitary
    "Pipes & Fittings": [
        "ppr", "pvc", "gi pipe", "مواسير", "كوع", "وصله", "fitting", "adapter", "connector"
    ],
    "Valves & Controls": [
        "valve", "gate valve", "check valve", "ball valve", "محبس"
    ],
    "Plumbing & Sanitary": [
        "tap", "mixer", "basin", "sink", "toilet", "wc", "shower", "faucet", "bidet", "حوض", "حنفيه"
    ],

    # Electrical & Lighting
    "Electrical": [
        "cable", "wire", "breaker", "panel", "سلك", "قاطع", "socket", "plug", "switch", "mcb", "db"
    ],
    "Lighting": [
        "lamp", "bulb", "led", "spotlight", "tube light", "fluorescent", "كشاف"
    ],

    # HVAC
    "HVAC": [
        "split ac", "duct", "مكيف", "فلكسبل", "hvac", "chiller", "fan coil", "grill", "diffuser"
    ],

    # Hardware & Tools
    "Fasteners & Hardware": [
        "screw", "bolt", "nut", "برغي", "مسامير", "washer", "anchor", "hinge"
    ],
    "Tools & Equipment": [
        "screwdriver", "drill", "cutter", "plier", "wrench", "hammer", "grinder", "saw", "مفك", "عدة"
    ],
    "Safety & PPE": [
        "helmet", "gloves", "mask", "ppe", "حذاء", "خوذه", "vest", "safety"
    ],

    # Chemicals & Misc
    "Adhesives & Sealants": [
        "glue", "adhesive", "sealant", "silicone", "epoxy", "لاصق"
    ],
    "Oils & Lubricants": [
        "grease", "oil", "lubricant", "wd40", "شحم"
    ],

    # Default (handled separately in function)
    "Other/Unknown": []
}


---
## Step3:
- Categorize items based on keyword matching.
- Handles missing or unknown descriptions, checks for category keywords and assigns "Others" if no match is found.

---

In [8]:
import re

def categorize_item(text, categories):
    # Handle missing item descriptions first
    if not isinstance(text, str) or text.strip().lower() == "unknown item":
        return "Unknown"
    
    # Check keywords
    for category, keywords in categories.items():
        for kw in keywords:
            if re.search(rf"\b{re.escape(kw)}\b", text):
                return category
    
    # Default if no keyword matched
    return "Others"

In [11]:
df["rule_category"] = df["Clean Item Name"].apply(lambda x: categorize_item(x, categories))

# Quick check
df[["Clean Item Name", "rule_category"]].sample(20)

,Clean Item Name,rule_category
2916,kitchen scissor,Others
2633,حديد تسليح اتفاق 16مم,Steel & Rebar
439,تيوب المنيوم مستطيل 20 80 1.30 ملي 6 متر,Others
2819,زجاج ستركشر مخفي و زجاج مزدوج,Others
2036,silica sand 0.1 1.2 mm 1.5 mt j b,Others
375,coil hr sabic 1.80x1000mm slitting 140,Sheets & Metals
645,حديد تسليح وطني 16مم 12متر,Steel & Rebar
3055,motor power 2 hp model no. hc t505,Others
423,pu foam 8mm thick as per previous supply un...,Others
1357,كيبل رياض 4 95 عادي,Others


In [12]:
df["rule_category"].value_counts(dropna=False)

rule_category
Others                  1581
Steel & Rebar            664
Unknown                  240
Pipes & Fittings         196
Sheets & Metals          169
Electrical               156
Fasteners & Hardware      50
HVAC                      19
Cement & Gypsum           12
Lighting                  11
Plumbing & Sanitary       11
Valves & Controls         11
Paints & Coatings         10
Wood & Panels              8
Oils & Lubricants          8
Adhesives & Sealants       3
Tools & Equipment          1
Name: count, dtype: int64

In [6]:
df.head()

,Item ID,Item Name,Quantity,Total Bcy,Purchase Order ID,Currency Code,Account ID,Total Spend in SAR,Unit Price,Clean Item Name,rule_category
0,3263831000000089042,Unknown Item,75.21,227510.25,3263831000000089040,SAR,3263831000000034003,227510.25,3025.0,unknown item,Unknown
1,3263831000000089046,Unknown Item,15.00,47250.00,3263831000000089040,SAR,3263831000000034003,47250.00,3150.0,unknown item,Unknown
2,3263831000000089044,Unknown Item,50.00,160000.00,3263831000000089040,SAR,3263831000000034003,160000.00,3200.0,unknown item,Unknown
3,3263831000000092044,Unknown Item,12.00,38280.00,3263831000000092040,SAR,3263831000000034003,38280.00,3190.0,unknown item,Unknown
4,3263831000000092052,Unknown Item,2.00,5900.00,3263831000000092040,SAR,3263831000000034003,5900.00,2950.0,unknown item,Unknown


---
## NOTE:
- Rule-based categorization was attempted using keyword matching across predefined categories.
- Despite iterative refinement and expansion of keyword rules, over 50% of items still fall into the "Others" bucket.
- This indicates poor coverage and diminishing returns from manual rule additions.
- Decision: Discontinue rule-based approach due to low scalability and semantic limitations.
- Next step: Explore embedding-based clustering or ML classification for improved generalization and coverage.

---